# Exercise 4.2

In [85]:
using NBInclude
using Dates
using LinearAlgebra,Plots
using OrdinaryDiffEq
using Parameters # @unpack


@nbinclude("./SAT_Const.ipynb")
@nbinclude("./SAT_Kepler.ipynb")
@nbinclude("./SAT_VecMat.ipynb")

R_z (generic function with 1 method)

- Satellite orbits (154.pag)

Exercise 4.2 (4th-Order Gauss–Jackson Method) Implement the 4th-order
Gauss–Jackson method (cf. Sect. 4.2.7) and integrate the second-order differen-
tial equation r̈ = −r/r 3 of the normalized two-body problem from t 0 = 0 to
t = 20 for an eccentricity of e = 0.1 (Test problem D1, cf. Sect. 4.1.6). De-
termine the resulting accuracy of the state vector at the end point using n =
{100, 300, 600, 1000, 1500, 2000, 3000, 4000} steps and count the correspond-
ing number of function evaluations.

## 4th-Order Gauss–Jackson Method

In [86]:
function  GJ4P(problem,n,k=1)  
    
     @unpack u0,tspan,p,f=problem
     t0,t1=tspan
     T=eltype(u0)
    
     m=4 # order of the method
     r0=u0[1:3]
     v0=u0[1:3]
     
#        
     h = (t1-t0)/(n*m)

#    Init GJ4P
     D=Array{Array{T,1}}(undef, m)        #Backward differences of acceleration at t
     d=Array{Array{T,1}}(undef, m)        #Backward differences of acceleration at t+h
     S1=zeros(3)
     S2=zeros(3)
     r_p=zeros(3)
     v_p=zeros(3)
     for i in 1:m 
        D[i]=zeros(3) 
        d[i]=zeros(3)
     end 
    
     GJ4P_Init!(problem,h,D,S1,S2)
     nfcn=16
    
     gp=[1, 1/2, 5/12, 3/8, 251/720]
     dp=[1, 0, 1/12,1/12,19/240,3/40]
    
#
    
     tt =  Array{T}(undef,n+1)
     uu = Array{Array{T,1}}(undef,n+1)

    #  
     tj = t0
     rj=copy(r0)
     vj=copy(v0)
     tt[1]=t0
     uu[1]=copy(u0)    
    

    for j in 1:n
        for i in 1:k
          GJ4PStep!(f,tj,rj,vj,p,h,d,D,S1,S2,gp,dp,r_p,v_p) 
          nfcn+=1
          tj = tj + h
        end
        tt[j+1] = tj
        uu[j+1] = copy(vcat(rj,vj))
    end
    
    println("steps=",n,",h=", h,",nfcn=", nfcn)
    DiffEqBase.build_solution(problem,GJ4P,tt,uu,retcode= :Success)

    
  end


function GJ4PStep!(f,tj,rj,vj,p,h,d,D,S1,S2,gp,dp,r_p,v_p)    

    
#     4th order predictor
     m=4

     @. r_p = dp[1]*S2
     for i in 3:m+2
        @. r_p =r_p+ dp[i]*D[i-2]
     end
     @. r_p = (h*h)*r_p
    
     @. v_p = gp[1]*S1
     for i in 2:m+1
       @. v_p =v_p+ gp[i]*D[i-1]
     end
     @. v_p =h*v_p
    
    
#    Update backwards difference table

     f(d[1],r_p,v_p,p,tj+h)            # Acceleration at t+h
     for i in 2:m
            @. d[i]=d[i-1]-D[i-1]        # New differences at t+h
     end
     for i in 1:m
           D[i].=d[i]                  # Update differences 
     end
     @. S1 =S1+d[1]
     @. S2 =S2+S1                            # Update sums

#    Update independent variable and solution

#      tj = tj + h  kanpoan
      rj .= r_p
      vj .= v_p
   
      return nothing
end         


function GJ4P_Init!(problema,h,D,S1,S2)
    
     @unpack u0,tspan,p,f=problema
     t0,T=tspan
     m=4
    
     # Coefficients of Moulton/Cowell correcto method    
     gc=[1, -1/2, -1/12, -1/24, -19/720]
     dc=[1, -1, 1/12, 0,-1/240, -1/240]
    
     tj_=t0
     r0=u0[1:3]
     v0=u0[4:6]
     rj_=copy(r0)
     vj_=copy(v0)
     
     # Create table of accelerations at past time t-3h, t-2h and t-h
     # using RK4 Steps

     f(D[1],rj_,vj_,p,tj_)
       
     for i in 2:m      # D[i+1]=a(t-ih)       
        RK4Step!(f,tj_,rj_,vj_,p,-h)
        tj_=tj_-h
        f(D[i],rj_,vj_,p,tj_)
     end

    
     # Compute backwards differences
    
     for i in 2:m
        for j in m:i
            @. D[j]=D[j-1]-D[j]
        end       
     end
    
     # Initialize backwards sums using 4th order GJ corrector
    
     @. S1=v0/h
     for i in 2:m+1
        @. S1=S1-gc[i]*D[i-1]
     end

     @. S2=r0/(h*h)-dc[2]*S1
     for i in 3:m+2
        @. S2=S2-dc[i]*D[i-2]
     end
 

     return nothing
end

GJ4P_Init! (generic function with 1 method)

In [87]:
function RK4Step!(f,t,r,v,p,h)
"""
 4th order Runge-Kutta step for 2nd order differential equation   
"""    

  T=eltype(r)
  vv=Array{Array{T,1}}(undef, 4)   # egokiago kanpoan sortzea
  aa=Array{Array{T,1}}(undef, 4)   # egokiago kanpoan sortzea

  for i in 1:4 
        vv[i]=zeros(3) 
        aa[i]=zeros(3)
  end 
    
  vv[1] .= v

  f(aa[1],r,v,p,t) 
  @. vv[2] = v+h/2*aa[1]
  f(aa[2], r+h/2*vv[1], vv[2],p, t+h/2)
  @. vv[3] = v+h/2*aa[2] 
  f(aa[3], r+h/2*vv[2], vv[3],p, t+h/2)
  @. vv[4] = v+h*aa[3]
  f(aa[4], r+h*vv[3] , vv[4],p, t+h)

  
#  t = t + h    kanpoan egin
  @. r = r + h/6*( vv[1] + 2*vv[2] + 2*vv[3] + vv[4] )
  @. v = v + h/6*( aa[1] + 2*aa[2] + 2*aa[3] + aa[4] )

  
  return nothing

end

RK4Step! (generic function with 1 method)

## Main Program

In [88]:
function f_Kep6D!(du,r,v,p,t)

# Second order derivative !!!
    
  du[1:3].=-r/norm(r)^3

  return nothing
    
end

f_Kep6D! (generic function with 1 method)

In [89]:
GM    = 1.0            # Gravitational coefficient
e     = 0.1            # Eccentricity
t_end = 20.0           # End time
Kep=[1.0,e,0.0,0.0,0.0,0.0]    # (a,e,i,Omega,omega,M)
y_ref = State(GM,Kep,t_end);    # Reference solution

tspan=(0.,t_end)
u0=[1.0-e,0.0,0.0, 0.0,sqrt((1+e)/(1-e)),0.0]
prob=ODEProblem(f_Kep6D!,u0,tspan,[]);

In [90]:
Steps= [100, 300, 600, 1000, 1500, 2000, 3000, 4000]
#h = t_end./Steps

for i in Steps
   h=t_end/i
   sol=GJ4P(prob,i)
   println(i, ", ", norm(y_ref-sol.u[end]))
end

steps=100,h=0.05,nfcn=116
100, 2.7660552298644028
steps=300,h=0.016666666666666666,nfcn=316
300, 2.7636711408450516
steps=600,h=0.008333333333333333,nfcn=616
600, 2.7638588930543877
steps=1000,h=0.005,nfcn=1016
1000, 2.764021688521462
steps=1500,h=0.0033333333333333335,nfcn=1516
1500, 2.764121800948472
steps=2000,h=0.0025,nfcn=2016
2000, 2.764176525038485
steps=3000,h=0.0016666666666666668,nfcn=3016
3000, 2.7642343565060212
steps=4000,h=0.00125,nfcn=4016
4000, 2.764264436385793
